# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20901994


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:02<00:40,  1.41s/it]

Rendering models:  10%|▉         | 3/31 [00:04<00:40,  1.45s/it]

Rendering models:  13%|█▎        | 4/31 [00:04<00:28,  1.06s/it]

Rendering models:  16%|█▌        | 5/31 [00:04<00:20,  1.26it/s]

...

Rendering models:  84%|████████▍ | 26/31 [00:11<00:00,  5.17it/s]

Rendering models:  87%|████████▋ | 27/31 [00:11<00:00,  5.67it/s]

Rendering models:  94%|█████████▎| 29/31 [00:11<00:00,  6.91it/s]

Rendering models: 100%|██████████| 31/31 [00:11<00:00,  7.58it/s]

equidad1                                0.000095
dianapenguinp                           0.000552
not-logged-in-3391886dcb0cd30b7163      0.007021
Melina_t                                0.000049
zoo1234565                              0.000171
jelstone20                              0.001373
Mark_Moffatt                            0.000501
kenyandall                              0.000660
jbowman0080                             0.001386
marius_aga                              0.000620
not-logged-in-9c310346c8df6cdafc65      0.066584
not-logged-in-9a45911eb71184fe7c7f      0.000666
clars915                                0.000081
ElisabethB                              0.000098
not-logged-in-5f30b2bb9b7f9d49da41      0.054117
raisamaduro                             0.000394
pangeli5                                0.000142
clairedeu                               0.000086
not-logged-in-214814e52004b42491a4    266.484062
not-logged-in-6d3e4eb12f2149553110      0.000121
acapirala           

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())